<a href="https://colab.research.google.com/github/sergio2pi/CEIA-FIUBA_LLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext
%pip install numpy pandas sentencepiece
%pip install transformers==4.16.2
%pip install pytorch-lightning==1.5.10

import pandas as pd
from sklearn.model_selection import train_test_split

Descarga del modelo

In [2]:
!git clone https://github.com/sergio2pi/CEIA-FIUBA_LLM.git
!git -C CEIA-FIUBA_LLM pull
!cp -r CEIA-FIUBA_LLM/t5 t5

Cloning into 'CEIA-FIUBA_LLM'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 0), reused 7 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 4.87 MiB | 17.56 MiB/s, done.
Already up to date.


Carga del dataset

In [3]:
data_path = "CEIA-FIUBA_LLM/dataset/medquad.csv"
data = pd.read_csv(data_path)

data.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16412 entries, 0 to 16411
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16412 non-null  object
 1   answer      16407 non-null  object
 2   source      16412 non-null  object
 3   focus_area  16398 non-null  object
dtypes: object(4)
memory usage: 513.0+ KB


El modelo recibe un dataframe con 2 columnas, el texto de origen ("source text") y el texto objetivo (target_text):

In [5]:
if 'source' in data.columns and 'focus_area' in data.columns:
  data.drop(columns=['source', 'focus_area'])
  data = data.rename(columns={"question":"source_text", "answer":"target_text"})
  data = data[['source_text', 'target_text']]

# Se elimina las filas con valores nulos
data = data.dropna()

data.head()

,source_text,target_text
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea..."
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ..."
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...


Dataset final

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16407 entries, 0 to 16411
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source_text  16407 non-null  object
 1   target_text  16407 non-null  object
dtypes: object(2)
memory usage: 384.5+ KB


In [7]:
train_df, test_df = train_test_split(data, test_size=0.2)
train_df.shape, test_df.shape

((13125, 2), (3282, 2))

Entrenamiento del modelo

In [8]:
import t5

model = t5.SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=50,
            target_max_token_len=150,
            batch_size=16,
            max_epochs=3,
            use_gpu=True,
            save_only_last_epoch=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [9]:
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-1.7911-val-loss-1.6506", use_gpu=True)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!cp -r outputs/ drive/MyDrive/IA/LLM/
!cp -r lightning_logs/ drive/MyDrive/IA/LLM/

In [12]:
input_text="What can cause heart failure?"
model.predict(input_text)

["Heart failure is caused by a combination of factors, including genetic and environmental factors. The most common cause of heart failure is diabetes mellitus (HDM). Diabetes mellitus is a condition that affects the heart's ability to pump blood through the body. In some cases, diabetes mellitus can be caused by mutations in the SLC1A gene. Other causes may include high blood pressure, low blood sugar, and/or hypertension."]

In [13]:
input_text="Can you tell me how to treat Alzheimer's disease?"
model.predict(input_text)

["How might Alzheimer's disease be treated? There is no cure for Alzheimer's disease, but it can be treated with medications and lifestyle changes. For example, if you have the condition, your doctor may prescribe medication to treat it. If you have the condition, your doctor may recommend that you take certain medications such as acetaminophen or corticosteroids. Your doctor may also recommend that you take anti-depressants (such as acetaminophen) in combination with other medicines."]

In [14]:
input_text="symptoms of Breast Cancer"
model.predict(input_text)

['Signs and symptoms of breast cancer are caused by breast cancer. Signs and symptoms of breast cancer may be caused by breast cancer or by other conditions. Signs and symptoms of breast cancer may be caused by breast cancer or by other conditions. Signs and symptoms of breast cancer may be caused by breast cancer or by other conditions. Signs and symptoms of breast cancer may be caused by breast cancer or by other conditions.']

In [15]:
input_text="Are Alopecia and breast pain symptoms of Breast Cancer?"
model.predict(input_text)

['Alopecia and breast pain are symptoms of breast cancer. They may be caused by a combination of factors, including the presence or absence of certain hormones in the body, such as estrogen, vitamin B12, and other hormones. These and other factors can affect the risk of developing alopecia and breast pain. In some cases, alopecia and breast pain is not a sign of breast cancer. However, it is possible to develop alopecia and breast pain without having any signs or symptoms.']